In [ ]:
import keys
import numpy as np
import pandas as pd
import json
import urllib
import requests
import re
import matplotlib.pyplot as plt
import requests
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import requests
from pprint import pprint
import json
%matplotlib inline

In [ ]:
#df2 = pd.read_csv('leads.csv')

In [ ]:
lead['lat'] = np.nan
lead['lng'] = np.nan

In [ ]:
df2['address'] = df2['Street'] + ' ' + df2['City'] + ' ' + \
    df2['State/Province'] + ' ' + df2['Zip/Postal Code']  # +' '+ df2['Country']

In [ ]:
df2[(df2['lat'].isnull() == True) &
    (df2['address'].isnull() == False) &
    (df2['Zip/Postal Code'].isnull() == False)
    ].head()

In [ ]:
#key = 'xxx'
key = keys.google
counter = 0
for i, row in df2[(df2['lat'].isnull() == True) &
                  (df2['address'].isnull() == False) &
                  (df2['Zip/Postal Code'].isnull() == False)
                  ].iterrows():
    address = row.address.replace(' ', '+')
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address='+address+'=&key='+key
    # print(url)
    # print(row.address)
    response = requests.get(url)
    try:
        data = json.loads(response.text)
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        df2.loc[df2['Lead ID'] == row['Lead ID'], 'lat'] = lat
        df2.loc[df2['Lead ID'] == row['Lead ID'], 'lng'] = lng
    except:
        #print ('request failure for row', i, row['Unnamed: 0'], data, url)
        pass
    counter += 1
    if counter % 500 == 0:
        print(counter, address, lat, lng)
    if counter % 500 == 0:
        df2.to_csv('lead geocoded.csv', index=False)
        # break

In [ ]:
df2.to_csv('lead geocoded.csv', index=False)

In [ ]:
print(df2[df2.lat.isnull() == True].shape)
print(df2[df2.lat.isnull() == False].shape)

In [ ]:
lead.to_csv('lead geocoded.csv')

In [ ]:
lead[(lead['lat'].isnull()) & (lead['lng'].isnull())].shape

In [ ]:
lead[lead['lat'].isnull() == False].shape

In [ ]:
df2 = pd.read_csv('lead geocoded.csv')

In [ ]:
df2.head(100)

In [ ]:
print(df2[df2.lat.isnull() == True].shape)
print(df2[df2.lat.isnull() == False].shape)

Many of the address without lats and lngs had faulty coiuntry information; these were corrected manually

In [ ]:
df2 = pd.read_csv('./sianedit_leads_with_distances.csv', encoding='ISO-8859-1')

In [ ]:
df2.head()

In [ ]:
df2.drop(['Unnamed: 0', 'City_1', 'lat_lng', 'closest_ts', 'within_one_tenth',
          'within_one_half', 'within_one', 'within_five'], axis=1, inplace=True)

In [ ]:
print(df2[df2.lat.isnull() == True].shape)
print(df2[df2.lat.isnull() == False].shape)

In [ ]:
print(df2[df2.address.isnull() == True].shape)
print(df2[df2.address.isnull() == False].shape)

In [ ]:
# rebuild address column
df2['address'] = df2['Street'] + ' ' + df2['City'] + ' ' + \
    df2['State/Province'] + ' ' + df2['Zip/Postal Code']+' ' + df2['Country']

In [ ]:
print(df2[df2.address.isnull() == True].shape)
print(df2[df2.address.isnull() == False].shape)

In [ ]:
# df2[df2.address.isnull()==True].head(100)

In [ ]:
print(df2[df2.address.isnull() == True].shape)
print(df2[df2.address.isnull() == False].shape)

In [ ]:
print(df2[df2.lat.isnull() == True].shape)
print(df2[df2.lat.isnull() == False].shape)

In [ ]:
final_leads = df2[df2.lat.isnull() == False]

In [ ]:
final_leads.to_csv('final_leads.csv', index_label=False)

In [ ]:
# get transitscreen info

In [ ]:
df = pd.read_csv('final_leads.csv')

In [ ]:
df['ts_response'] = np.nan
df['mobility_score'] = np.nan
df['carshare'] = np.nan
df['bikeshare'] = np.nan
df['ridehailing'] = np.nan
df['masstransit'] = np.nan

In [ ]:
df.head()

In [ ]:
base_url = 'http://mobilityscore-dev.transitscreen.io/api/v1/locations.json?&debug=true'
key = keys.ts

for i, row in df[(df['ts_response'].isnull() == True)
                 #& (df['id']=='0061a00000GcQdL') #for test
                 ].iterrows():
    #print(row.id, row.name, row.lat, row.lng)
    try:
        url = base_url + '&key='+key+'&geojson=true&coordinates=' + \
            str(row.lat)+','+str(row.lng)
        #(print(url))
        response = requests.get(url)  # wait up to 10 seconds
        data = json.loads(response.text)
        full_response = data['data']
        mobility_score = full_response['mobilityScore']['score']
        #print (mobility_score)
        raw_scores = full_response['mobilityScore']['rawScoreBreakdown']
        #print (raw_scores)
        carshare = raw_scores['carshare']
        bikeshare = raw_scores['bikeshare']
        ridehailing = raw_scores['ridehailing']
        masstransit = raw_scores['masstransit']
        #print(carshare, bikeshare, ridehailing, masstransit)
        df.loc[df['id'] == row.id, 'ts_response'] = data
        df.loc[df['id'] == row.id, 'mobility_score'] = mobility_score
        df.loc[df['id'] == row.id, 'carshare'] = carshare
        df.loc[df['id'] == row.id, 'bikeshare'] = bikeshare
        df.loc[df['id'] == row.id, 'ridehailing'] = ridehailing
        df.loc[df['id'] == row.id, 'masstransit'] = masstransit
    except:
        print('error on', i, raw_scores)
    if i % 100 == 0:
        df.to_csv('model_data.csv', index=False)
        print(i, raw_scores)